In [1]:
import torch
import torch.nn.functional as F
import numpy as np
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import datasets, transforms
from matplotlib import pyplot as plt
from torchsummary import summary

In [2]:
###   Load data

transformation = transforms.Compose (
    [
        transforms.CenterCrop ((500, 500)),
        transforms.Resize (224),
        transforms.ToTensor (),
    ]
)

train_data = datasets.Flowers102 ("data", split = "train", download = True, transform = transformation)
validation_data = datasets.Flowers102 ("data", split = "val", download = True, transform = transformation)
test_data = datasets.Flowers102 ("data", split = "test", download = True, transform = transformation)

100%|██████████| 344862509/344862509 [00:04<00:00, 85618692.45it/s]


Extracting data/flowers-102/102flowers.tgz to data/flowers-102


100%|██████████| 502/502 [00:00<00:00, 494258.36it/s]


100%|██████████| 14989/14989 [00:00<00:00, 6410566.19it/s]


In [3]:
###   Setup loader

phases = {
    'train': train_data,
    'valid': validation_data,
    'test': test_data
}

#   Parameters for loader
batch_size = 32

loader = {phase: DataLoader (dataset, batch_size = batch_size, shuffle = (phase == "train"))
    for phase, dataset in phases.items ()
}

In [4]:
###   Model evaluation function

def evaluate (model_, loader, device, criterion, mode = "valid"):
    '''Function for running model evaluation.'''

    model_.eval ()

    total = 0
    total_correct = 0
    total_loss = 0

    for i, (images, labels) in enumerate (loader[mode]):

        images = images.to (device)
        labels = labels.to (device)

        with torch.no_grad ():
            outputs = model_ (images)
            loss = criterion (outputs, labels)
            total_loss += loss.item () * images.size (0)
            total += images.size (0)
            _, predictions = outputs.max (1)
            total_correct += (labels == predictions).sum ()

    accuracy = total_correct / total
    loss = total_loss / total

    msg = f"Loss({loss:6.4f}) Accuracy({accuracy:6.4f})"

    if mode == "valid":
        print (f"{mode}: " + msg)
    else:
        print (f"{mode}: " + msg)

In [6]:
###   Model training function

def train (model_, loader, device, criterion, num_epochs = 10):
    '''Function for running model training.

    Using a function to easily repeat the running of the training process
    over multiple model without having to continuously re-enter the code.
    '''

    model_ = model_.to (device)

    for epoch in range (num_epochs):

        model_.train ()

        total = 0
        total_correct = 0
        total_loss = 0

        for i, (images, labels) in enumerate (loader['train']):
            images = images.to (device)
            labels = labels.to (device)

            optimizer.zero_grad ()

            outputs = model_ (images)

            loss = criterion (outputs, labels)
            loss.backward ()
            optimizer.step ()

            total += images.size (0)

            _, predictions = outputs.max (1)

            total_correct += (predictions == labels).sum ()
            total_loss += loss.item () * images.size (0)

        accuracy = total_correct / total
        loss = total_loss / total

        print (f"Epoch {epoch + 1}")
        print (f"Train: Loss({loss:6.4f}) Accuracy({accuracy:6.4f})")

        evaluate (model_, loader, device, criterion, mode = 'valid')

Adding an Adaptive Average Pool 2d function between the convolution layer and the classifier to have more control over the output size of the convolution layer.

In [7]:
###   Create model

class Model (nn.Module):
    '''Class defining the neural network.

    Parameters:
    num_classes: the number of possible classifications for the model
    '''

    def __init__ (self, num_classes = 1000):
        super (Model, self).__init__ ()

        self.features = nn.Sequential (
            nn.Conv2d (3, 32, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU (inplace = True),
            nn.MaxPool2d (kernel_size = 3, stride = 6),
            nn.Conv2d (32, 32, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU (inplace = True),
            nn.MaxPool2d (kernel_size = 3)
        )

        self.adaptive_avg_pool2d = nn.AdaptiveAvgPool2d ((12, 12))

        self.flatten = nn.Flatten (start_dim = 1)

        self.classifier = nn.Sequential (
            nn.Linear (32 * 12 * 12, 512),
            nn.ReLU (),
            nn.Dropout (0.5),
            nn.Linear (512, num_classes)
        )

    def forward (self, x):
        x = self.features (x)
        x = self.flatten (x)
        x = self.classifier (x)
        return x

In [8]:
###   Instantiate model

model = Model (num_classes = 102)

In [9]:
###   Model parameters

learning_rate = 0.01
momentum = 0.9   #   optimizer momentum

device = "cuda" if torch.cuda.is_available () else "cpu"

criterion = nn.CrossEntropyLoss ()
optimizer = torch.optim.SGD (model.parameters (), lr = learning_rate, momentum = momentum)

In [10]:
###   Train model

train (model, loader, device, criterion, num_epochs = 10)

Epoch 1
Train: Loss(4.6285) Accuracy(0.0078)
valid: Loss(4.6237) Accuracy(0.0078)
Epoch 2
Train: Loss(4.6239) Accuracy(0.0127)
valid: Loss(4.6211) Accuracy(0.0098)
Epoch 3
Train: Loss(4.6196) Accuracy(0.0059)
valid: Loss(4.6133) Accuracy(0.0216)
Epoch 4
Train: Loss(4.6033) Accuracy(0.0235)
valid: Loss(4.5791) Accuracy(0.0225)
Epoch 5
Train: Loss(4.5431) Accuracy(0.0225)
valid: Loss(4.4298) Accuracy(0.0373)
Epoch 6
Train: Loss(4.3457) Accuracy(0.0255)
valid: Loss(4.2438) Accuracy(0.0363)
Epoch 7
Train: Loss(4.2048) Accuracy(0.0353)
valid: Loss(4.0904) Accuracy(0.0735)
Epoch 8
Train: Loss(3.9364) Accuracy(0.0647)
valid: Loss(3.8842) Accuracy(0.0794)
Epoch 9
Train: Loss(3.8195) Accuracy(0.0951)
valid: Loss(3.8043) Accuracy(0.0961)
Epoch 10
Train: Loss(3.5721) Accuracy(0.1539)
valid: Loss(3.7025) Accuracy(0.1333)


In [11]:
evaluate (model, loader, device, criterion, mode = 'test')

test: Loss(3.7909) Accuracy(0.1000)


The model results in an accuracy of ~10 % with loss of 3.79 after 10 epochs.